In [1]:
using GLMakie
using Revise
include("EFITModule/EFIT.jl")


Main.EFIT

# Initialize files

In [2]:
bovs = Vector{String}()
dir = "BOVTests/"
for f in readdir(dir)
    fname, ftype = split(f,".")
    if ftype == "bov"
        push!(bovs,"$dir/$f")
    end
end
data = Main.EFIT.readBOV(bovs[1]);
print(size(data))

(434, 358, 716)

# Volumetric animation

In [3]:
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = volume(data,algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)
record(fig, "Animation_Volume.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.volume = data
end

"Animation_Volume.mp4"

# 2D Slices

## X Slice

In [4]:
depth = 217
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = heatmap(data[depth,:,:],colorrange = (0, 0.01),colormap=:curl)
record(fig, "Animation_XSlice.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.heatmap = data[depth,:,:]
end

"Animation_XSlice.mp4"

# Composite

In [20]:
xDepth = 217
yDepth = 177
zDepth = 358

data = Main.EFIT.readBOV(bovs[1]);

fig = Figure()

lscene = LScene(fig[1, 1], scenekw = (camera = cam3d!, raw = false))

fig[1,2] = Axis(fig, title = "X Slice")
fig[2,1] = Axis(fig, title = "Y Slice")
fig[2,2] = Axis(fig, title = "Z Slice")
crange =(0, 0.01)
cmap=:curl 

record(fig, "Animation_Composite.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    volume!(lscene, data,algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)
    heatmap!(fig[1,2],data[floor(Int,end/2),:,:],colorrange = crange,colormap=cmap)
    heatmap!(fig[2,1],data[:,floor(Int,end/2),:],colorrange = crange,colormap=cmap)
    heatmap!(fig[2,2],data[:,:,floor(Int,end/2)],colorrange = crange,colormap=cmap)

end

"Animation_Composite.mp4"